In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json

In [2]:
class_category = json.load(open('../data_files/bbox_labels_600_hierarchy(1).json'))
class_names = pd.read_csv('../data_files/class-descriptions-boxable.csv', header=None, names=['class', 'name'])

In [3]:
print(class_category['Subcategory'])

[{'LabelName': '/m/0242l'}, {'LabelName': '/m/03120'}, {'LabelName': '/m/0h8l4fh'}, {'LabelName': '/m/0138tl', 'Subcategory': [{'LabelName': '/m/0167gd'}, {'LabelName': '/m/01j51'}, {'LabelName': '/m/029b3'}, {'LabelName': '/m/02wmf'}, {'LabelName': '/m/02zt3'}, {'LabelName': '/m/0kmg4'}]}, {'LabelName': '/m/019dx1', 'Subcategory': [{'LabelName': '/m/0174k2'}, {'LabelName': '/m/01k6s3'}, {'LabelName': '/m/029bxz'}, {'LabelName': '/m/02pjr4'}, {'LabelName': '/m/02wv84t'}, {'LabelName': '/m/02x984l'}, {'LabelName': '/m/03qhv5'}, {'LabelName': '/m/03s_tn'}, {'LabelName': '/m/03wvsk'}, {'LabelName': '/m/040b_t'}, {'LabelName': '/m/04169hn'}, {'LabelName': '/m/04lvq_'}, {'LabelName': '/m/063rgb'}, {'LabelName': '/m/07xyvk'}, {'LabelName': '/m/0fx9l'}, {'LabelName': '/m/0ky7b'}, {'LabelName': '/m/0llzx'}, {'LabelName': '/m/0c3m8g'}, {'LabelName': '/m/03ldnb'}]}, {'LabelName': '/m/02pkr5', 'Subcategory': [{'LabelName': '/m/0130jx'}, {'LabelName': '/m/01vbnl'}, {'LabelName': '/m/02f9f_'}, {'La

In [4]:
print(class_names.head())

         class        name
0    /m/011k07    Tortoise
1  /m/011q46kg   Container
2    /m/012074      Magpie
3    /m/0120dh  Sea turtle
4    /m/01226z    Football


In [5]:
print(len(class_names))

601


In [6]:
class_category_copy = class_category.copy()

def convert_label_to_class(label):
    for i, row in class_names.iterrows():
        if row['class'] == label:
            return row['name']

def replace_label_names(d):
    if 'LabelName' in d:
        d['LabelName'] = convert_label_to_class(d['LabelName']) 
    if 'Subcategory' in d:
        for sub in d['Subcategory']:
            replace_label_names(sub)
    if 'Part' in d:
        for part in d['Part']:
            replace_label_names(part)

replace_label_names(class_category_copy)

In [7]:
print(class_category_copy['Subcategory'])
with open('./new_labels.json', 'w') as f:
    json.dump(class_category_copy, f, indent=4)

[{'LabelName': 'Coin'}, {'LabelName': 'Flag'}, {'LabelName': 'Light bulb'}, {'LabelName': 'Toy', 'Subcategory': [{'LabelName': 'Doll'}, {'LabelName': 'Balloon'}, {'LabelName': 'Dice'}, {'LabelName': 'Flying disc'}, {'LabelName': 'Kite'}, {'LabelName': 'Teddy bear'}]}, {'LabelName': 'Home appliance', 'Subcategory': [{'LabelName': 'Washing machine'}, {'LabelName': 'Toaster'}, {'LabelName': 'Oven'}, {'LabelName': 'Blender'}, {'LabelName': 'Gas stove'}, {'LabelName': 'Mechanical fan'}, {'LabelName': 'Heater'}, {'LabelName': 'Kettle'}, {'LabelName': 'Hair dryer'}, {'LabelName': 'Refrigerator'}, {'LabelName': 'Wood-burning stove'}, {'LabelName': 'Humidifier'}, {'LabelName': 'Mixer'}, {'LabelName': 'Coffeemaker'}, {'LabelName': 'Microwave oven'}, {'LabelName': 'Dishwasher'}, {'LabelName': 'Sewing machine'}, {'LabelName': 'Hand dryer'}, {'LabelName': 'Ceiling fan'}]}, {'LabelName': 'Plumbing fixture', 'Subcategory': [{'LabelName': 'Sink'}, {'LabelName': 'Bidet'}, {'LabelName': 'Shower'}, {'Lab

Convert ground truth sub class to father class

In [8]:
class_cat_new = json.load(open('./new_labels.json'))

In [9]:
parent_map = {}

def get_immediate_parent(class_name, parent_map):
    return parent_map.get(class_name) if class_name in parent_map and parent_map[class_name] else class_name

def build_parent_map(d, parent=None):
    # Handle if data is a list (multiple top-level nodes)
    if isinstance(d, list):
        for item in d:
            build_parent_map(item, parent)
        return
    class_name = d['LabelName']
    parent_map[class_name] = parent
    if 'Subcategory' in d:
        for sub in d['Subcategory']:
            build_parent_map(sub, class_name)
    if 'Part' in d:
        for part in d['Part']:
            build_parent_map(part, class_name)

build_parent_map(class_cat_new)

In [10]:
with open('./parent_map.json', 'w') as f:
    json.dump(parent_map, f, indent=4)

In [11]:
print(parent_map)

{None: None, 'Coin': None, 'Flag': None, 'Light bulb': None, 'Toy': None, 'Doll': 'Toy', 'Balloon': 'Toy', 'Dice': 'Toy', 'Flying disc': 'Sports equipment', 'Kite': 'Toy', 'Teddy bear': 'Bear', 'Home appliance': None, 'Washing machine': 'Home appliance', 'Toaster': 'Home appliance', 'Oven': 'Kitchen appliance', 'Blender': 'Kitchen appliance', 'Gas stove': 'Home appliance', 'Mechanical fan': 'Home appliance', 'Heater': 'Home appliance', 'Kettle': 'Home appliance', 'Hair dryer': 'Home appliance', 'Refrigerator': 'Kitchen appliance', 'Wood-burning stove': 'Home appliance', 'Humidifier': 'Home appliance', 'Mixer': 'Kitchen appliance', 'Coffeemaker': 'Kitchen appliance', 'Microwave oven': 'Kitchen appliance', 'Dishwasher': 'Kitchen appliance', 'Sewing machine': 'Home appliance', 'Hand dryer': 'Home appliance', 'Ceiling fan': 'Home appliance', 'Plumbing fixture': None, 'Sink': 'Plumbing fixture', 'Bidet': 'Plumbing fixture', 'Shower': 'Plumbing fixture', 'Tap': 'Plumbing fixture', 'Bathtub':

In [12]:
import ast

ground_truth = pd.read_csv('./bowv/filtered_ground_truth.csv')
ground_truth['Description'] = ground_truth['Description'].apply(lambda x: ast.literal_eval(x))

In [13]:
print(ground_truth.head())

            ImageID                                        Description
0  0000253ea4ecbf19  [Carnivore, Human leg, Mammal, Plant, Tree, Hu...
1  0000271195f2c007  [Human eye, Headphones, Boy, Person, Human mou...
2  0000286a5c6a3eb5  [Human eye, Sunglasses, Shorts, Person, Human ...
3  00003bfccf5f36c2  [Person, Lantern, Chair, Table, Tree, Furnitur...
4  000045257f66b9e2  [Boy, Person, Cowboy hat, Hat, Fedora, Human b...


In [14]:
def get_top_level_parent(class_name, parent_map):
    """Find the top-level parent of a class."""
    current = class_name
    while current in parent_map and parent_map[current] is not None:
        current = parent_map[current]
    return current

def convert_to_top_level_classes(description_list, parent_map):
    """Convert all classes in the list to their top-level parents and return unique list."""
    top_level_set = set()
    for class_name in description_list:
        top_level = get_immediate_parent(class_name, parent_map)
        top_level_set.add(top_level)
    return list(top_level_set)

In [15]:
ground_truth['Top Level Classes'] = ground_truth['Description'].apply(lambda x: convert_to_top_level_classes(x, parent_map))

In [16]:
print(ground_truth.head())

            ImageID                                        Description  \
0  0000253ea4ecbf19  [Carnivore, Human leg, Mammal, Plant, Tree, Hu...   
1  0000271195f2c007  [Human eye, Headphones, Boy, Person, Human mou...   
2  0000286a5c6a3eb5  [Human eye, Sunglasses, Shorts, Person, Human ...   
3  00003bfccf5f36c2  [Person, Lantern, Chair, Table, Tree, Furnitur...   
4  000045257f66b9e2  [Boy, Person, Cowboy hat, Hat, Fedora, Human b...   

                                   Top Level Classes  
0                [Plant, Mammal, Animal, Human body]  
1         [Headphones, Person, Human body, Clothing]  
2  [Human body, Clothing, Person, Animal, Persona...  
3  [Window, Lantern, Furniture, Plant, Building, ...  
4  [Human body, Hat, Clothing, Trousers, Plant, F...  


In [17]:
ground_truth.to_csv('./ground_truth_top_level.csv', index=False)

In [58]:
from neo4j import GraphDatabase

URI = "neo4j://localhost:7687"  
USERNAME = "neo4j"            
PASSWORD = "Tai123321"     

In [59]:
def get_neo4j_driver():
    return GraphDatabase.driver(URI, auth=(USERNAME, PASSWORD))

In [75]:
#add is_category_of relationship between parent and child
def create_graph_from_json(json_data, session):
    for item in json_data:
        label_name = item['LabelName']
        if label_name:
            # Create a node for the main label
            session.run("MERGE (n:Class {name: $name})", name=label_name)

            # If there are subcategories, create them as well
            if 'Subcategory' in item:
                for sub in item['Subcategory']:
                    sub_label_name = sub['LabelName']
                    if sub_label_name:
                        session.run(
                            "MATCH (a:Class {name: $parent_name}) "
                            "MERGE (a)-[:HAS_SUBCATEGORY]->(b:Class {name: $sub_name})",
                            parent_name=label_name,
                            sub_name=sub_label_name
                        )
                    # If this subcategory has its own subcategories
                    if 'Subcategory' in sub:
                        create_graph_from_json(sub['Subcategory'], session)

create_graph_from_json(class_cat_new['Subcategory'], get_neo4j_driver().session())

In [61]:
print(class_cat_new)

{'LabelName': None, 'Subcategory': [{'LabelName': 'Coin'}, {'LabelName': 'Flag'}, {'LabelName': 'Light bulb'}, {'LabelName': 'Toy', 'Subcategory': [{'LabelName': 'Doll'}, {'LabelName': 'Balloon'}, {'LabelName': 'Dice'}, {'LabelName': 'Flying disc'}, {'LabelName': 'Kite'}, {'LabelName': 'Teddy bear'}]}, {'LabelName': 'Home appliance', 'Subcategory': [{'LabelName': 'Washing machine'}, {'LabelName': 'Toaster'}, {'LabelName': 'Oven'}, {'LabelName': 'Blender'}, {'LabelName': 'Gas stove'}, {'LabelName': 'Mechanical fan'}, {'LabelName': 'Heater'}, {'LabelName': 'Kettle'}, {'LabelName': 'Hair dryer'}, {'LabelName': 'Refrigerator'}, {'LabelName': 'Wood-burning stove'}, {'LabelName': 'Humidifier'}, {'LabelName': 'Mixer'}, {'LabelName': 'Coffeemaker'}, {'LabelName': 'Microwave oven'}, {'LabelName': 'Dishwasher'}, {'LabelName': 'Sewing machine'}, {'LabelName': 'Hand dryer'}, {'LabelName': 'Ceiling fan'}]}, {'LabelName': 'Plumbing fixture', 'Subcategory': [{'LabelName': 'Sink'}, {'LabelName': 'Bide